# Robust Knapsack

This tutorial shows how to construct and solve the Robust Portfolio problem, introduced by Bertsimas and Sim 2004 [1] (https://pubsonline.informs.org/doi/abs/10.1287/opre.1030.0065)
We show to how solve the robust knapsack problem (Section 6.1). Using the same formulation as in [1], the problem can be formulated as follows:
$$
\begin{align}
\max_{\mathbf{x}} &\;\mathbf{c}^T \mathbf{x} \nonumber \\

\text{s.t. } &\mathbf{\tilde{w}}^T\mathbf{x} \leq b \nonumber \\

&\mathbf{x}\in\left\{ 0,1\right\}^n \nonumber \\

&\mathbf{w}-\pmb{\delta} \leq \mathbf{\tilde{w}} \leq \mathbf{w}+\pmb{\delta} \\
\end{align}
$$

where there are $n$ items, $\mathbf{x}$ are the binary decision variables, their values are denoted by $\mathbf{c}$, and their weights $\mathbf{\tilde{w}}$ belong to a box uncertainty set, where the expected weights are denoted by $\mathbf{w}$, and their uncertainties are captured by $\pmb{\delta}$.

In [1]:
import cvxpy as cp
import numpy as np

from lropt import Budget, Box
from lropt.robust_problem import RobustProblem
from lropt.uncertain import UncertainParameter

np.random.seed(seed=1234)

We define the constants as shown below:

In [2]:
n = 200 #Number of items
b = 1000 #Capacity
c = np.random.uniform(low=0., high=1., size=n) #Value of each item
w = np.random.uniform(low=1., high=2, size=n) #Mean weight of each item
delta = np.random.uniform(low=0., high=0.1, size=n) #Weights uncertainties

The uncertain parameter $\mathbf{\tilde{w}}$ is formulated using LROPT in the block below. We use the box uncertainty set, which is defined as follows:

Box uncertainty: $\{ u \mid\| Au + b\|_\infty \leq \rho \}$ 

- $\rho$ : float, optional  
  * Box scaling. Default 1.0.
- $A$ : np.array, optional
  * Scaling matrix for u. Default identity matrix.
- $b$ : np.array, optional
  * Relocation vector for u. Default None.

We define a helper uncertain variable $\mathbf{u}$ that satisfies $-\pmb{\delta}\leq\mathbf{u}\leq\pmb{\delta}$, so the uncertain paramater $\mathbf{\tilde{w}}$ is defined as $\mathbf{\tilde{w}} = \mathbf{w} + \mathbf{u}$.

In [4]:
#Define a helper uncertainty variable u
uncertainty_set = Box(rho=1, a=np.diag(1/delta))
u = UncertainParameter(n, uncertainty_set=uncertainty_set)
w_hat = w+u
x = cp.Variable(n, boolean=True) #Optimization variable

#Define and solve the problem
objective = cp.Maximize(c@x)
constraints = [
                w_hat@x <= b,
              ]
prob = RobustProblem(objective=objective, constraints=constraints)
prob.solve()

105.50851379393394